In [2]:
# Cell [1]: Setup & Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, ParameterGrid
from sklearn.metrics import mean_squared_error, r2_score

# Import your RoBoost PLSR model from the local package
# Make sure roboost_plsr is a folder with __init__.py and roboost_plsr.py
# containing the class RoBoostPLSR.
from roboost_plsr.roboost_plsr import RoBoostPLSR

import time  # For timing certain operations


In [ ]:
# Cell [2]: Data Loading & Preparation

# Path to the CSV dataset. Adjust if necessary.
# For example, if this notebook is in 'examples/' and CSV is in 'examples/data/',
# then the relative path is 'data/DATASET.csv'.
DATA_PATH = 'data/DATASET.csv'

# Load the dataset
data_new = pd.read_csv(DATA_PATH, sep=';')

# Create a 'Color' column based on 'Variety'
data_new['Color'] = data_new['Variety'].apply(lambda x: 'white' if x == 'MAUZAC' else 'red')

# Drop rows with missing values
data_new.dropna(inplace=True)

# Separate target (Sugar content) and features
y_new = data_new['Sugar content (g/l)']
X_new = data_new.drop(columns=['Unnamed: 0', 'Variety', 'Sugar content (g/l)', 'Color'])

# Extract wavelengths from column names
wavelengths_float = [float(x.split('x.')[1]) for x in X_new.columns]

# Subsets by color (optional)
white_grapes = data_new[data_new['Color'] == 'white']
red_grapes = data_new[data_new['Color'] == 'red']
varieties = data_new['Variety'].unique()  # List of grape varieties

In [ ]:
# Cell [3]: Exploratory Data Analysis (EDA)

# A. Plot spectral data by color (sample 5 rows)
for color in data_new['Color'].unique():
    plt.figure(figsize=(8, 4))
    subset = X_new[data_new['Color'] == color].sample(n=5, random_state=42)
    for index, row in subset.iterrows():
        plt.plot(wavelengths_float, row, label=f'Sample {index}')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Reflectance')
    plt.title(f'Spectral Data for {color.capitalize()} Grapes')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# B. Plot spectral data by variety (sample 5 rows)
for variety in varieties:
    plt.figure(figsize=(8, 4))
    subset = X_new[data_new['Variety'] == variety].sample(n=5, random_state=42)
    for index, row in subset.iterrows():
        plt.plot(wavelengths_float, row, label=f'Sample {index}')
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Reflectance')
    plt.title(f'Spectral Data for {variety} Grapes')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# C. Distribution of sugar content by color
for color in data_new['Color'].unique():
    plt.figure(figsize=(6, 4))
    sns.histplot(
        data=data_new[data_new['Color'] == color],
        x='Sugar content (g/l)',
        kde=True,
        color='blue' if color == 'white' else 'red',
        bins=20
    )
    plt.xlabel('Sugar content (g/l)')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Sugar Content for {color.capitalize()} Grapes')
    plt.legend([color.capitalize()])
    plt.tight_layout()
    plt.show()

# D. Boxplot of sugar content by variety
for variety in varieties:
    plt.figure(figsize=(8, 4))
    sns.boxplot(
        x='Variety',
        y='Sugar content (g/l)',
        data=data_new[data_new['Variety'] == variety]
    )
    plt.xlabel('Variety')
    plt.ylabel('Sugar content (g/l)')
    plt.title(f'Sugar Content Distribution for {variety} Grapes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# E. Correlation matrix of X_new
plt.figure(figsize=(10, 10))
sns.heatmap(X_new.corr(), cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Spectral Bands')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Wavelength (nm)')
plt.tight_layout()
plt.show()

In [ ]:
# Cell [4]: Principal Component Analysis (PCA)

scaler_pca = StandardScaler()
X_new_scaled_pca = scaler_pca.fit_transform(X_new)

pca_new = PCA()
principal_components_new = pca_new.fit_transform(X_new_scaled_pca)

explained_variance_new = pca_new.explained_variance_ratio_
cumulative_variance_new = np.cumsum(explained_variance_new)

# Scree Plot
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(explained_variance_new) + 1), cumulative_variance_new,
         marker='o', linestyle='--')
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.96, '95% Threshold', color='red', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

# Determine # of components for 95% variance
n_components_to_keep = np.argmax(cumulative_variance_new >= 0.95) + 1
print(f'Number of components to keep for 95% explained variance: {n_components_to_keep}')

# Plot the first two principal components
pc_df_new = pd.DataFrame(
    principal_components_new[:, :2],
    columns=['PC1', 'PC2']
)
pc_df_new['SugarContent'] = y_new.values

plt.figure(figsize=(6, 4))
scatter = plt.scatter(
    pc_df_new['PC1'],
    pc_df_new['PC2'],
    c=pc_df_new['SugarContent'],
    cmap='viridis'
)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA - First Two Principal Components')
cbar = plt.colorbar(scatter)
cbar.set_label('Sugar content (g/l)')
plt.tight_layout()
plt.show()

# Heatmap of PCA loadings
loadings_new = pca_new.components_.T[:, :n_components_to_keep]
loadings_df_new = pd.DataFrame(
    loadings_new,
    index=wavelengths_float,
    columns=[f'PC{i+1}' for i in range(n_components_to_keep)]
)

plt.figure(figsize=(8, 6))
sns.heatmap(loadings_df_new, cmap='coolwarm', center=0)
plt.title('Heatmap of Principal Component Loadings')
plt.xlabel('Principal Components')
plt.ylabel('Wavelength (nm)')
plt.tight_layout()
plt.show()

In [ ]:
# Cell [5]: Train/Test Split

X_train, X_test, y_train, y_test = train_test_split(
    X_new,
    y_new,
    test_size=0.2,
    random_state=42
)

In [ ]:
# Cell [6]: PCR

pcr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=n_components_to_keep)),
    ('lr', LinearRegression())
])
pcr_pipeline.fit(X_train, y_train)

# Predictions
y_pred_pcr = pcr_pipeline.predict(X_test)
mse_pcr = mean_squared_error(y_test, y_pred_pcr)
r2_pcr = r2_score(y_test, y_pred_pcr)

print(f'PCR MSE: {mse_pcr:.4f}')
print(f'PCR R^2: {r2_pcr:.4f}')

# Cross-validation for PCR
cv = KFold(n_splits=5, shuffle=True, random_state=42)
mse_cv_pcr = -cross_val_score(pcr_pipeline, X_new, y_new,
                              cv=cv, scoring='neg_mean_squared_error').mean()
r2_cv_pcr = cross_val_score(pcr_pipeline, X_new, y_new,
                            cv=cv, scoring='r2').mean()

print(f'PCR Cross-Validated MSE: {mse_cv_pcr:.4f}')
print(f'PCR Cross-Validated R^2: {r2_cv_pcr:.4f}')

In [ ]:
# Cell [7]: PLSR

def optimize_plsr(X, y, max_components=20):
    """
    Optimize the number of PLS components via cross-validation.
    Returns (optimal_n_components, mse_list, r2_list).
    """
    mse_list = []
    r2_list = []
    component_range = range(1, max_components + 1)

    for n_comp in component_range:
        pls = PLSRegression(n_components=n_comp)
        mse_cv = -cross_val_score(pls, X, y,
                                  cv=cv,
                                  scoring='neg_mean_squared_error').mean()
        r2_cv = cross_val_score(pls, X, y,
                                cv=cv,
                                scoring='r2').mean()
        mse_list.append(mse_cv)
        r2_list.append(r2_cv)

    plt.figure(figsize=(10, 5))
    plt.plot(component_range, mse_list, marker='o', label='MSE')
    plt.xlabel('Number of PLS Components')
    plt.ylabel('Cross-Validated MSE')
    plt.title('PLSR: Number of Components vs. MSE')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Minimizing MSE
    optimal_n_comp = component_range[np.argmin(mse_list)]
    print(f'Optimal number of PLS components: {optimal_n_comp}')

    return optimal_n_comp, mse_list, r2_list

# Optimize # of components
optimal_n_plsr, mse_plsr_cv, r2_plsr_cv = optimize_plsr(X_new, y_new, max_components=20)

pls_new_optimal = PLSRegression(n_components=optimal_n_plsr)
pls_new_optimal.fit(X_train, y_train)

y_pred_plsr = pls_new_optimal.predict(X_test).flatten()
mse_plsr = mean_squared_error(y_test, y_pred_plsr)
r2_plsr = r2_score(y_test, y_pred_plsr)

print(f'PLSR MSE: {mse_plsr:.4f}')
print(f'PLSR R^2: {r2_plsr:.4f}')

mse_cv_plsr = -cross_val_score(pls_new_optimal, X_new, y_new,
                               cv=cv,
                               scoring='neg_mean_squared_error').mean()
r2_cv_plsr = cross_val_score(pls_new_optimal, X_new, y_new,
                             cv=cv,
                             scoring='r2').mean()

print(f'PLSR Cross-Validated MSE: {mse_cv_plsr:.4f}')
print(f'PLSR Cross-Validated R^2: {r2_cv_plsr:.4f}')

In [ ]:
# Cell [8]: RoBoost PLSR Hyperparameter Optimization

def optimize_roboost_plsr(X, y, ncomp, param_grid, niter=50, th=1 - 1e-12):
    """
    Optimize alpha, beta, gamma for RoBoostPLSR using cross-validation.
    Returns (best_params, results_df).
    """
    best_params = None
    best_mse = np.inf
    best_r2 = -np.inf
    results = []

    # We'll reuse the KFold 'cv' from earlier
    grid = list(ParameterGrid(param_grid))
    total_combinations = len(grid)
    start_time = time.time()

    for idx, params in enumerate(grid):
        alpha = params['alpha']
        beta = params['beta']
        gamma = params['gamma']

        mse_fold = []
        r2_fold = []
        skip_combination = False

        for train_idx, val_idx in cv.split(X):
            X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
            y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]

            # Scale data
            scaler_cv = StandardScaler()
            X_train_cv_scaled = scaler_cv.fit_transform(X_train_cv)
            X_val_cv_scaled = scaler_cv.transform(X_val_cv)

            X_train_cv_df = pd.DataFrame(X_train_cv_scaled, columns=X.columns)
            X_val_cv_df = pd.DataFrame(X_val_cv_scaled, columns=X.columns)

            roboost = RoBoostPLSR(ncomp=ncomp,
                                  niter=niter,
                                  gamma=gamma,
                                  beta=beta,
                                  alpha=alpha,
                                  th=th)
            roboost.fit(X_train_cv_df, y_train_cv)

            if not roboost.fit_successful:
                print(f"Skipping combo: alpha={alpha}, beta={beta}, gamma={gamma}")
                skip_combination = True
                break

            y_pred_cv = roboost.predict(X_val_cv_df)
            mse_fold.append(mean_squared_error(y_val_cv, y_pred_cv))
            r2_fold.append(r2_score(y_val_cv, y_pred_cv))

        if skip_combination:
            results.append({'alpha': alpha, 'beta': beta, 'gamma': gamma,
                            'MSE': np.nan, 'R2': np.nan})
            continue

        avg_mse = np.mean(mse_fold)
        avg_r2 = np.mean(r2_fold)
        results.append({'alpha': alpha, 'beta': beta, 'gamma': gamma,
                        'MSE': avg_mse, 'R2': avg_r2})

        # Track best combo
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_r2 = avg_r2
            best_params = params

        elapsed_time = time.time() - start_time
        avg_time_per_combo = elapsed_time / (idx + 1)
        remaining_time = avg_time_per_combo * (total_combinations - (idx + 1))
        print(f"Completed {idx+1}/{total_combinations} combos. "
              f"Estimated {remaining_time:.2f} seconds remaining")

    results_df = pd.DataFrame(results)
    print("Cross-Validation Results for RoBoost PLSR:")
    print(results_df)

    if best_params is not None:
        print(f"\nBest Params: alpha={best_params['alpha']}, "
              f"beta={best_params['beta']}, gamma={best_params['gamma']}")
        print(f"Best Cross-Validated MSE: {best_mse:.4f}")
        print(f"Best Cross-Validated R²: {best_r2:.4f}")
    else:
        print("\nNo valid parameter combo found.")

    return best_params, results_df

# Step 1: Fix alpha=beta=gamma=inf, optimize ncomp
print("\n--- Step 1: Optimize ncomp (alpha=beta=gamma=inf) ---")
param_grid_roboost = {
    'alpha': [1, 2, 4, 6, np.inf],
    'beta': [1, 2, 4, 6, np.inf],
    'gamma': [1, 2, 4, 6, np.inf]
}
optimal_ncomp_step1, _, _ = optimize_plsr(X_new, y_new, max_components=20)

# Step 2: Optimize alpha, beta, gamma with the best ncomp
print("\n--- Step 2: Optimize alpha, beta, gamma (fixed ncomp) ---")
best_params_roboost, results_roboost = optimize_roboost_plsr(
    X_new, y_new,
    ncomp=optimal_ncomp_step1,
    param_grid=param_grid_roboost,
    niter=50,
    th=1 - 1e-12
)

# Train final RoBoost model
scaler_roboost = StandardScaler()
X_train_scaled_roboost = scaler_roboost.fit_transform(X_train)
X_test_scaled_roboost = scaler_roboost.transform(X_test)

X_train_roboost_df = pd.DataFrame(X_train_scaled_roboost, columns=X_new.columns)
X_test_roboost_df = pd.DataFrame(X_test_scaled_roboost, columns=X_new.columns)

roboost_plsr = RoBoostPLSR(
    ncomp=optimal_ncomp_step1,
    niter=best_params_roboost.get('niter', 50),
    gamma=best_params_roboost['gamma'],
    beta=best_params_roboost['beta'],
    alpha=best_params_roboost['alpha'],
    th=1 - 1e-12
)
roboost_plsr.fit(X_train_roboost_df, y_train)

y_pred_roboost = roboost_plsr.predict(X_test_roboost_df)
mse_roboost = mean_squared_error(y_test, y_pred_roboost)
r2_roboost = r2_score(y_test, y_pred_roboost)

print(f'RoBoost PLSR MSE: {mse_roboost:.4f}')
print(f'RoBoost PLSR R^2: {r2_roboost:.4f}')

In [ ]:
# Cell [9]: Model Comparison

results = pd.DataFrame({
    'Model': ['PCR', 'PLSR', 'RoBoost PLSR'],
    'MSE': [mse_pcr, mse_plsr, mse_roboost],
    'R^2': [r2_pcr, r2_plsr, r2_roboost],
    'Cross-Validated MSE': [mse_cv_pcr, mse_cv_plsr, np.round(mse_roboost)],
    'Cross-Validated R^2': [r2_cv_pcr, r2_cv_plsr, np.round(r2_roboost)]
})

print("\nComparison of Models:")
print(results)

# R² and MSE
plt.figure(figsize=(10, 4))

# R²
plt.subplot(1, 2, 1)
sns.barplot(x='Model', y='R^2', data=results, palette='viridis')
plt.title('Comparison of R² Scores')
plt.ylim(0, 1)
plt.ylabel('R² Score')

# MSE
plt.subplot(1, 2, 2)
sns.barplot(x='Model', y='MSE', data=results, palette='magma')
plt.title('Comparison of MSE Scores')
plt.ylabel('MSE Score')

plt.tight_layout()
plt.show()

# Cross-Validated R²
plt.figure(figsize=(7, 6))
sns.barplot(x='Model', y='Cross-Validated R^2', data=results, palette='viridis')
plt.title('Comparison of Cross-Validated R² Scores')
plt.ylim(0, 1)
plt.ylabel('Cross-Validated R² Score')
plt.tight_layout()
plt.show()

# Cross-Validated MSE
plt.figure(figsize=(7, 6))
sns.barplot(x='Model', y='Cross-Validated MSE', data=results, palette='magma')
plt.title('Comparison of Cross-Validated MSE Scores')
plt.ylabel('Cross-Validated MSE Score')
plt.tight_layout()
plt.show()